The two following lines stand for automatic reload of Jupyter when modification are done on this page. 

In [ ]:
%reload_ext autoreload
%autoreload 2

The following lines stand for library tools. If a plot is does not displayed correctly then not add the line "%matplotlib inline". To display the help of very useful commends, type "%magic".

In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import os
import imp
import shutil

_, path, _ = imp.find_module('ImSimpy')

# 1) Compute the SNR of a star with the Exposure Time Calculator (ETC)



We want to calculate the Signal to Noise Ratio (SNR) for the following star:
    
    Magnitude: 16
    Photometric system: AB

The telescope is :

    Site: San Perdo Martir
    Telescope: COLIBRI
    Optics: COLIBRI Near Infrared arm. filter J
    Camera: CAGIRE 
    Exposure: 7s (=5*1.4s)

Write the hjson configuration file [Make_one_image_etc.hjson](../ImSimpy/configFiles/Make_one_image_etc.hjson) in the folder ImSimpy/configFiles that contains all the input parameters of the computation of the SNR.

First import the package

In [ ]:
from pyETC.pyETC import etc

Load the hjson configuration file. If no configfile is given, it will load the default one "default.hjson". os.getenv get the environment variable define by the key ImSimpy_DIR to select the path according to the location of gft-sim.

In [ ]:
colibri_etc=etc(configFile='Make_one_image_etc.hjson',name_telescope='colibri')

In [ ]:
colibri_etc.sim()

The following file [info.txt](../../gft_etc/gft_etc/results/info.txt) in the folder gft-sim/gft_etc/gft_etc/results 
contains very detailed results. 

To retreive only the integrated SNR calculated:

In [ ]:
colibri_etc.information['SNR']

To retreive all results calculated:

In [ ]:
colibri_etc.information

# 2) Simulate one image for COLIBRI

## 2.1) Generate maps for instrumental effects (offset, gain map, hot pixels,...)

In [ ]:
# Name of output_dir. Will be used for creating folder in catalog/, images/, etc...
output_dir='Make_one_image'

In [ ]:
from ImSimpy.utils.generateCalib import Offset, Vignetting, GainMap

Offset(filename=path+'/data/Offset/%s/Offset_vis.fits' % output_dir,
     xsize=4096, ysize=4096, floor=1000)
Offset(filename=path+'/data/Offset/%s/Offset_nir.fits' % output_dir,
     xsize=2048, ysize=2048, floor=500)

Vignetting(filename=path+'/data/Vignetting/%s/Vignetting_vis.fits' % output_dir,
     floor=1,xsize=4096, ysize=4096)
Vignetting(filename=path+'/data/Vignetting/%s/Vignetting_nir.fits' % output_dir,
     floor=1,xsize=2048, ysize=2048)

GainMap(filename=path+'/data/GainMap/%s/Gain_vis.fits' % output_dir,
        Type='constant',mean=1.5,xsize=4096,ysize=4096)
GainMap(filename=path+'/data/GainMap/%s/Gain_nir.fits' % output_dir,
        Type='random',mean=1.3, std=0.1,Nampl=32,xsize=2048,ysize=2048)


In [ ]:
from ImSimpy.utils.generateCosmetics import HotPixs, DeadPixs

HotPixs(filename=path+'/data/Cosmetics/%s/HotPixs_nir.fits' % output_dir,
        xsize=2048,ysize=2048,Type='random',hotpixels=500,hotmin=5000,hotmax=40000)
HotPixs(filename=path+'/data/Cosmetics/%s/HotPixs_vis.fits' % output_dir,
        xsize=4096,ysize=4096,Type='random',hotpixels=500,hotmin=70000,hotmax=300000)

DeadPixs(filename=path+'/data/Cosmetics/%s/DeadPixs_nir.fits' % output_dir,
         xsize=2048,ysize=2048,Type='random',deadpixels=500)
DeadPixs(filename=path+'/data/Cosmetics/%s/DeadPixs_vis.fits' % output_dir,
         xsize=4096,ysize=4096,Type='random',deadpixels=2000)

## 2.2) Generate raw image

Write the hjson configuration file [Make_one_image_is.hjson](../../gft_is/gft_is/configFiles/Make_one_image_is.hjson) in the folder ImSimpy/configFiles that contains all the input parameters of the computation of the image.

First import the package

In [ ]:
from ImSimpy.ImSimpy import ImageSimulator

In [ ]:
colibri_IS=ImageSimulator(configFile='Make_one_image_is.hjson',
                          name_telescope='colibri')

In [ ]:
colibri_IS.simulate()

To retreive the image name simulated:

In [ ]:
fits_filename=path+'/images/'+colibri_IS.information['output']
print (fits_filename)

To display the calculated image:

In [ ]:
from astropy.io import fits
import numpy as np

In [ ]:
fits_file = fits.getdata(fits_filename)
image_mean = np.mean(fits_file)
image_std = np.std(fits_file)
vmin = image_mean - 5*image_std
vmax = image_mean + 10*image_std
plt.figure(figsize=(7,7))
plt.imshow(fits_file,interpolation='none',cmap='gray',vmin=vmin,vmax=vmax)

Add wcs coordinates

In [ ]:
from astropy.wcs import WCS

In [ ]:
image,header = fits.getdata(fits_filename,header=True)

wcs = WCS(header)
plt.figure(figsize=(7,7))

plt.subplot(projection=wcs)
plt.imshow(image,interpolation='none',cmap='gray',vmin=vmin,vmax=vmax,origin='lower')
plt.grid(color='white', ls='solid')
plt.xlabel('Right Ascension (J2000)')
plt.ylabel('Declination (J2000)')